In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

Below code builds basic block of ResNet.

It contains 2 layers. Each layer is consist of conv --> Batchnorm --> ReLU.

At end input of first layer is fed to output of second layer before ReLU.

Downsampling is applicable only if nomber of output channels of current block are not equal to number of channels of next block.

In [2]:
# Define the basic block of ResNet
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):  # by default downsample is None
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample  # Used for dimension matching in residual connections

    def forward(self, x):
        identity = x               # input
        if self.downsample:
            identity = self.downsample(x)
        out = self.conv1(x)           # first conv layer of block
        out = self.bn1(out)           # batch norm
        out = F.relu(out)             #RelU
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity               # Concatination before applyinf ReLU
        out = F.relu(out)
        return out

In [3]:
# Define the ResNet-34 model
class ResNet34(nn.Module):
    def __init__(self, num_classes=1000):
        super(ResNet34, self).__init__()
        self.in_channels = 64

        # Initial convolution and max-pooling
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Residual blocks
        self.layer1 = self._make_layer(64, 3, stride=1)  # output channels, number of blocks, stride
        self.layer2 = self._make_layer(128, 4, stride=2)
        self.layer3 = self._make_layer(256, 6, stride=2)
        self.layer4 = self._make_layer(512, 3, stride=2)

        # Global Average Pooling and Fully Connected Layer
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Averages each feature map to single value, 1 value for each channel
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, blocks, stride):
        """Create a residual block."""
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
            )
        layers = [BasicBlock(self.in_channels, out_channels, stride, downsample)]
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(BasicBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [4]:
# Data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [5]:
# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root="./data", train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [00:14<00:00, 11.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
for i in train_loader:
  batch_index = 0
  print(i[0][batch_index].shape)
  print(i[1][batch_index])
  break

torch.Size([3, 224, 224])
tensor(9)


In [7]:
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Metrics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(dataloader)
    accuracy = 100.0 * correct / total
    return epoch_loss, accuracy


def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(dataloader)
    accuracy = 100.0 * correct / total
    return epoch_loss, accuracy


In [8]:
# Hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet34(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

from torchsummary import summary

summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
            Conv2d-7           [-1, 64, 56, 56]          36,864
       BatchNorm2d-8           [-1, 64, 56, 56]             128
        BasicBlock-9           [-1, 64, 56, 56]               0
           Conv2d-10           [-1, 64, 56, 56]          36,864
      BatchNorm2d-11           [-1, 64, 56, 56]             128
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
       BasicBlock-14           [-1, 64,

In [9]:
num_epochs = 10

for epoch in range(num_epochs):
    train_loss, train_accuracy = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")


Epoch 1/10
Train Loss: 1.6702, Train Accuracy: 38.17%
Test Loss: 1.4674, Test Accuracy: 46.55%
Epoch 2/10
Train Loss: 1.0162, Train Accuracy: 63.80%
Test Loss: 0.9160, Test Accuracy: 67.20%
Epoch 3/10
Train Loss: 0.7169, Train Accuracy: 74.86%
Test Loss: 0.6956, Test Accuracy: 75.99%
Epoch 4/10
Train Loss: 0.5652, Train Accuracy: 80.40%
Test Loss: 0.6032, Test Accuracy: 79.25%
Epoch 5/10
Train Loss: 0.4608, Train Accuracy: 83.97%
Test Loss: 0.6893, Test Accuracy: 77.48%
Epoch 6/10
Train Loss: 0.3758, Train Accuracy: 86.81%
Test Loss: 0.5689, Test Accuracy: 81.32%
Epoch 7/10
Train Loss: 0.2981, Train Accuracy: 89.62%
Test Loss: 0.6046, Test Accuracy: 80.96%
Epoch 8/10
Train Loss: 0.2282, Train Accuracy: 91.96%
Test Loss: 0.5885, Test Accuracy: 81.48%
Epoch 9/10
Train Loss: 0.1631, Train Accuracy: 94.31%
Test Loss: 0.6119, Test Accuracy: 82.95%
Epoch 10/10
Train Loss: 0.1253, Train Accuracy: 95.60%
Test Loss: 0.6561, Test Accuracy: 82.48%
